# Split-Operator Fourier Transform (SOFT) program

In [ ]:
%%html
<style>
.mytext > .widget-label {
    font-style: bold;
    color: black;
    font-size: 18px;
}
.mytext > input[type="text"] {
    font-size: 20px;
    color: red;
}
</style>

In [ ]:
from widget_code_input import WidgetCodeInput

code_widget = WidgetCodeInput(
    function_name = "generate_potential_data",
    function_parameters = "num",
    docstring="""
    A function to generate the potential data array.
    You need return a numeric array or list for the potential.  
    
    :num is the number of the points. 
    """,
    
    function_body="# Input here your solution\n",
    
    code_theme = 'midnight'
)

#display(code_widget)

<hr size="5">
<font size=3px>
<h2>1$^{st}$ step: define the potential</h2>
<ol>
  <li>Box potential.</li>
  <li>Morse potential.</li>
  <li>Harmonic potential.</li>
  <li>Define your custom potential from the function shown below.</li>
</ol> 


In [ ]:
import ipywidgets as widgets
import os
import subprocess
from time import *
from ipywidgets import HBox, VBox, HTML, FloatSlider, IntSlider, IntProgress, Label, Checkbox 
import shlex
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from IPython.display import display, clear_output
from IPython.display import Image
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import webbrowser


class MyHandler(FileSystemEventHandler):
    def on_modified(self, event):
        sleep(0.1)
        fprogress = open("./progress/count.dat");
        line = fprogress.readlines()[0]
        progress_bar.value = int(line.split()[0])
        progress_precentage.value = str(progress_bar.value) + "%"
        v_norm.value = "Norm: " + str(line.split()[1])
        v_energy.value = "Total energy: " + str(line.split()[2])
        fprogress.close()
        
        with out:
            fpot = open("potential.dat","r");
            lines = fpot.readlines();

            x1 = [];
            y1 = [];

            for line in lines:
                if len(line.split()) == 3:
                    a = line.split()[0]
                    b = line.split()[1]
                    c = line.split()[2]
                    x1.append(float(b))
                    y1.append(float(c)/10.0)

            fpsi = open("psi_new.dat", "r");
            lines = fpsi.readlines();

            a1 = [];
            b1 = [];
            c1 = [];
            d1 = [];

            for line in lines:
                a = line.split()[1]
                b = line.split()[2]
                c = line.split()[3]
                d = line.split()[4]
                a1.append(float(a))
                b1.append(float(b))
                c1.append(float(c))
                d1.append(float(d))    

            fpot.close()
            fpsi.close()

            out.clear_output(wait=True)
            fig = plt.figure(figsize=(14,6))

            plt.plot(x1, y1, 'green',label='potential');
            if psi_real.value == True:
                plt.plot(a1, b1, 'b', label="$\Psi$ Real");
            if psi_image.value == True:
                plt.plot(a1, c1, 'r', label="$\Psi$ Image"); 
            if psi_squre.value == True:
                plt.plot(a1, d1, 'k', label="$\Psi^2$");
            plt.xlim([0, 50])
            plt.ylim([-0.6, 0.6])


            plt.grid()
            plt.legend(loc = 'lower center', fontsize=20, ncol=4)
            plt.show()

event_handler = MyHandler()
observer = Observer()
observer.schedule(event_handler, path='./progress/', recursive=False)
observer.start()


out = widgets.Output()
out2 = widgets.Output()

def on_savefig(b):
    p = subprocess.check_call('tail -512 psi.dat > final_psi.dat', shell=True)
    fig1 = plt.figure(figsize=(14,6))

    fpot = open("potential.dat","r");
    lines = fpot.readlines();

    x1 = [];
    y1 = [];

    for line in lines:
        if len(line.split()) == 3:
            a = line.split()[0]
            b = line.split()[1]
            c = line.split()[2]
            x1.append(float(b))
            y1.append(float(c)/10.0)

    plt.plot(x1, y1, 'green',label='potential')

    fpsi = open("final_psi.dat", "r");
    lines = fpsi.readlines();

    a1 = [];
    b1 = [];
    c1 = [];
    d1 = [];

    for line in lines:
        a = line.split()[1]
        b = line.split()[2]
        c = line.split()[3]
        d = line.split()[4]
        a1.append(float(a))
        b1.append(float(b))
        c1.append(float(c))
        d1.append(float(d))

    if psi_real.value == True:
        plt.plot(a1, b1, 'b', label="Psi Real");
    if psi_image.value == True:
        plt.plot(a1, c1, 'r', label="Psi Image");
    if psi_squre.value == True:
        plt.plot(a1, d1, 'k', label="Psi Square");
    plt.xlim([0, 50])
    plt.ylim([-0.6, 0.6])


    plt.grid()
    plt.legend(fontsize=20)
    plt.savefig("test.pdf")
    webbrowser.open('./test.pdf')


def run_soft(pot_type, dt, mass, pt, bh, bw, vb, vD):
    command = "./SOFT.x " + str(pot_type) + ' ' + str(dt);
    command = command + ' ' + str(mass) + ' ' + str(pt) + ' ' + str(bh) + ' ' + str(bw);
    command = command + ' ' + str(vb) + ' ' + str(vD);
    p = subprocess.check_call(command, shell=True)
    
            
def show_animation(b):
    with out2:
        out2.clear_output()
        show_button.style.button_color = 'red';
        p = subprocess.check_call('./make_gif.sh', shell=True)
        display(Image(filename='./result.gif'))
        show_button.style.button_color = 'green';
        
def run_soft_button(b):
    run_button.style.button_color = 'red'
    if w.index == 4:
        f = open("my_potential.dat","w");
        from math import sin
        user_function = code_widget.get_function_object()

        for i in user_function(512):
            f.write(str(i))
            f.write("\n")

        f.close()
        
    out2.clear_output()
    run_soft(w.index, dt.value, mass.value, pt.value, BH.value, BW.value, vb.value, vD.value)
    run_button.style.button_color = 'green'


style = {'description_width': 'initial'}

w = widgets.Dropdown(
    options=['None', '1. Box potential', '2. Morse potential', '3. Harmonic potential','4. Read potential from your file'],
    value='None',
    description='Potential type:',
    disabled=False,
    style = style
)

w.add_class("mytext"); 

def on_value_change(b):
    if w.index == 1:
        BH.layout = layout_visible
        BW.layout = layout_visible
        vb.layout = layout_hidden
        vD.layout = layout_hidden
        code_widget.layout = layout_hidden
    elif w.index == 2:
        vb.layout = layout_visible
        vD.layout = layout_visible
        BH.layout = layout_hidden
        BW.layout = layout_hidden
        code_widget.layout = layout_hidden
    elif w.index == 3:
        vb.layout = layout_visible
        vD.layout = layout_visible
        BH.layout = layout_hidden
        BW.layout = layout_hidden
        code_widget.layout = layout_hidden
    elif w.index == 4:
        code_widget.layout = layout_visible 
        vb.layout = layout_hidden
        vD.layout = layout_hidden
        BH.layout = layout_hidden
        BW.layout = layout_hidden
    else:
        BH.layout = layout_hidden
        BW.layout = layout_hidden
        vb.layout = layout_hidden
        vD.layout = layout_hidden
        code_widget.layout = layout_hidden


run_button = widgets.Button(description='Run SOFT')
save_button = widgets.Button(description='Save figure')
show_button = widgets.Button(description='Show animation')
progress_bar = IntProgress(value=0, min=0, max=100, bar_style='success')
progress_precentage = widgets.Label('')

run_button.style.button_color = 'green';
save_button.style.button_color = 'green';
show_button.style.button_color = 'green';

save_button.on_click(on_savefig)
show_button.on_click(show_animation)
run_button.on_click(run_soft_button)

layout_hidden  = widgets.Layout(visibility = 'hidden')
layout_visible = widgets.Layout(visibility = 'visible')


BH = IntSlider(value=0, min=0, max=10, description='BH');
BW = IntSlider(value=0, min=0, max=10, description='BW');

vb = FloatSlider(value=0.3, min=0.1, max=1.0, description='b');
vD = FloatSlider(value=1.0, min=0.1, max=2.0, description='D');

BH.layout = layout_hidden
BW.layout = layout_hidden
vb.layout = layout_hidden
vD.layout = layout_hidden
code_widget.layout = layout_hidden


v_norm = Label(value=' ');
v_energy = Label(value=' ');

psi_real = widgets.Checkbox(value=True, description="$\Psi$ Real");
psi_image = widgets.Checkbox(value=True, description="$\Psi$ Image");
psi_squre = widgets.Checkbox(value=True, description="$\Psi^2$");

cbox = HBox([psi_real, psi_image, psi_squre]);


dt = FloatSlider(value=0.01, min=0.01, max=0.10, step=0.01, description='dt')
mass = IntSlider(value=1, min=1, max=10, description='mass')
pt = IntSlider(value=1, min=0, max=10, description='Pause time per step', style=style)
# dt.observe(on_value_change, names='value', type='change')
w.observe(on_value_change, names='value', type='change')
display(VBox([w, code_widget, BH, BW, vb, vD]))

<hr size="5">
<font size=3px>
<h2>2$^{nd}$ step: choice the simulation parameters</h2>

In [ ]:
display(VBox([dt, mass, pt]))

## 

<hr size="5">
<font size=3px>
<h2>3$^{rd}$ step: define the intial wavefunction</h2>
<ol>
  <li>Define a Gaussian type wavefunction and its parameters. </li>
  <li>Define a custom initial wavefunction (You need to normalize the function).</li>
</ol>  
    
<hr size="5">

In [ ]:
display(VBox([HBox([run_button, progress_bar, progress_precentage]), HBox([v_norm, v_energy]), cbox, out, HBox([save_button, show_button]),out2]))